In [28]:
import pandas as pd
from matplotlib import pyplot as plt
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
from sklearn.metrics import accuracy_score
import datetime
from catboost import CatBoostRegressor
from geopy import distance
import statsmodels.api as sm
from scipy import stats
from math import *

In [4]:
Train = pd.read_csv("Train.csv")
Test = pd.read_csv("Test.csv")
Rider = pd.read_csv("Riders.csv")
sp = pd.read_csv("SampleSubmission.csv")

In [25]:
def compute_distance(row):
  src = (row["Pickup Lat"],row["Pickup Long"])
  dest= (row["Destination Lat"],row["Destination Long"])
  return distance.distance(src,dest).km
Train["geopy_distance"] = Train.apply(compute_distance,axis=1)
Test["geopy_distance"] = Test.apply(compute_distance,axis=1)

In [6]:
# ORS
ors_test = pd.read_csv("ors_test.csv")
ors_train = pd.read_csv("ors_train.csv")

In [7]:
Train["distance_ors"] = ors_train["distance"]/1000  # en mètre / sec
Test["distance_ors"] = ors_test["distance"]/1000
Train["duration_ors"] = ors_train["duration"]  # sec
Test["duration_ors"] = ors_test["duration"]

In [8]:
Train = pd.merge(Train, Rider, how='left', left_on='Rider Id', right_on='Rider Id', left_index=True)
Test = pd.merge(Test, Rider, how='left', left_on='Rider Id', right_on='Rider Id', left_index=True)

In [9]:
col_x = ["Platform Type","Personal or Business","Pickup Long","Pickup Lat","Destination Long","Destination Lat","Distance (KM)","Pickup - Weekday (Mo = 1)","Pickup - Time","No_Of_Orders","Age","Average_Rating","distance_ors","duration_ors"]
col_x = sorted(col_x)
col_y = ["Time from Pickup to Arrival"]
Train = Train[col_x+col_y]
Test = Test[col_x]

In [10]:
def convert_time24(nom,col,data=Train):
  tab = []
  for i in col:
    s = i.split()
    terminaison = s[1]
    if terminaison == "PM":
      time = s[0].split(":")
      time = (int(time[0]) + 12) * 3600 + int(time[1]) * 60 + int(time[2])
      tab.append(time)
    else:
      time = s[0].split(":")
      time = (int(time[0]) + 12) * 3600 + int(time[1]) * 60 + int(time[2])
      tab.append(time)
  data[nom] = tab
  return True

In [11]:
# Train
#convert_time24("Placement - Time",Train["Placement - Time"])
#convert_time24("Confirmation - Time",Train["Confirmation - Time"])
#convert_time24("Pickup - Time",Train["Pickup - Time"])

In [12]:
#Test
#convert_time24("Placement - Time",Test["Placement - Time"],Test)
#convert_time24("Confirmation - Time",Test["Confirmation - Time"],Test)
#convert_time24("Pickup - Time",Test["Pickup - Time"],Test)

In [13]:
#Train['Temperature'] = Train['Temperature'].fillna(Train['Temperature'].mean())
#Test['Temperature'] = Test['Temperature'].fillna(Test['Temperature'].mean())
#Train['Precipitation in millimeters'] = Train['Precipitation in millimeters'].fillna(Train['Precipitation in millimeters'].mean())
#Test['Precipitation in millimeters'] = Test['Precipitation in millimeters'].fillna(Test['Precipitation in millimeters'].mean())
#Test['Precipitation in millimeters'] = Test['Precipitation in millimeters'].fillna(0)

In [14]:
def heure(data):
  time=[]
  data = pd.to_datetime(data)
  for i in data:
      i = i.time()
      ti = (str(i).split(':')[0])
      time.append(int(ti))
  time
  data = time
  return data

In [15]:
Train["Pickup - Time"] = heure(Train ['Pickup - Time'])
Test['Pickup - Time'] = heure(Test ['Pickup - Time'])

In [16]:
def dummy(dataset,columns):
  for c in columns:
    dum = pd.get_dummies(dataset[c],prefix=c,prefix_sep="_")
    dataset = pd.concat([dataset,dum], axis=1)
    del dataset[c]
  return dataset

In [17]:
dummies_col = ["Platform Type","Personal or Business","Pickup - Weekday (Mo = 1)"]

In [18]:
Train = dummy(Train,dummies_col)
Test =dummy(Test,dummies_col)

In [19]:
y_train = Train['Time from Pickup to Arrival']
c = list(set(Train.columns) - {"Time from Pickup to Arrival"})
X_train = Train[sorted(c)]
X_test = Test[sorted(c)]

In [20]:
from lightgbm import LGBMRegressor

In [21]:
X_train['Personal or Business_Business'] = X_train['Personal or Business_Business'].astype(float)
X_train['Personal or Business_Personal'] = X_train['Personal or Business_Personal'].astype(float)
X_train['Pickup - Weekday (Mo = 1)_1'] = X_train['Pickup - Weekday (Mo = 1)_1'].astype(float)
X_train['Pickup - Weekday (Mo = 1)_2'] = X_train['Pickup - Weekday (Mo = 1)_2'].astype(float)
X_train['Pickup - Weekday (Mo = 1)_3'] = X_train['Pickup - Weekday (Mo = 1)_3'].astype(float)
X_train['Pickup - Weekday (Mo = 1)_4'] = X_train['Pickup - Weekday (Mo = 1)_4'].astype(float)
X_train['Pickup - Weekday (Mo = 1)_5'] = X_train['Pickup - Weekday (Mo = 1)_5'].astype(float)
X_train['Pickup - Weekday (Mo = 1)_6'] = X_train['Pickup - Weekday (Mo = 1)_6'].astype(float)
X_train['Pickup - Weekday (Mo = 1)_7'] = X_train['Pickup - Weekday (Mo = 1)_7'].astype(float)
X_train['Platform Type_1'] = X_train['Platform Type_1'].astype(float)
X_train['Platform Type_2'] = X_train['Platform Type_2'].astype(float)
X_train['Platform Type_3'] = X_train['Platform Type_3'].astype(float)
X_train['Platform Type_4'] = X_train['Platform Type_4'].astype(float)

/snap/jupyter/6/lib/python3.7/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.
/snap/jupyter/6/lib/python3.7/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
/snap/jupyter/6/lib/python3.7/site-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats i

In [22]:
X_test['Pickup - Weekday (Mo = 1)_1'] = X_test['Pickup - Weekday (Mo = 1)_1'].astype(float)
X_test['Pickup - Weekday (Mo = 1)_2'] = X_test['Pickup - Weekday (Mo = 1)_2'].astype(float)
X_test['Pickup - Weekday (Mo = 1)_3'] = X_test['Pickup - Weekday (Mo = 1)_3'].astype(float)
X_test['Pickup - Weekday (Mo = 1)_4'] = X_test['Pickup - Weekday (Mo = 1)_4'].astype(float)
X_test['Pickup - Weekday (Mo = 1)_5'] = X_test['Pickup - Weekday (Mo = 1)_5'].astype(float)
X_test['Pickup - Weekday (Mo = 1)_6'] = X_test['Pickup - Weekday (Mo = 1)_6'].astype(float)
X_test['Pickup - Weekday (Mo = 1)_7'] = X_test['Pickup - Weekday (Mo = 1)_7'].astype(float)
X_test['Platform Type_1'] = X_test['Platform Type_1'].astype(float)
X_test['Platform Type_2'] = X_test['Platform Type_2'].astype(float)
X_test['Platform Type_3'] = X_test['Platform Type_3'].astype(float)
X_test['Platform Type_4'] = X_test['Platform Type_4'].astype(float)

In [30]:
model = CatBoostRegressor(loss_function='RMSE',
                         iterations=2030, learning_rate=0.05, depth=5)
model.fit(X_train,y_train)

0:	learn: 972.0802956	total: 8.64ms	remaining: 17.5s
1:	learn: 957.3726869	total: 14.1ms	remaining: 14.3s
2:	learn: 944.0721387	total: 20.2ms	remaining: 13.7s
3:	learn: 931.9327764	total: 26.1ms	remaining: 13.2s
4:	learn: 920.6513626	total: 31ms	remaining: 12.5s
5:	learn: 910.4285200	total: 35.8ms	remaining: 12.1s
6:	learn: 901.0270047	total: 40.7ms	remaining: 11.8s
7:	learn: 892.3055469	total: 45.5ms	remaining: 11.5s
8:	learn: 884.4776189	total: 50.7ms	remaining: 11.4s
9:	learn: 876.8200967	total: 57.7ms	remaining: 11.6s
10:	learn: 869.4005474	total: 63ms	remaining: 11.6s
11:	learn: 863.2968106	total: 67.8ms	remaining: 11.4s
12:	learn: 857.5815499	total: 72.7ms	remaining: 11.3s
13:	learn: 852.3554503	total: 77.4ms	remaining: 11.1s
14:	learn: 847.3323778	total: 83.7ms	remaining: 11.2s
15:	learn: 842.9090120	total: 88.5ms	remaining: 11.1s
16:	learn: 838.7908265	total: 98.3ms	remaining: 11.6s
17:	learn: 834.3023367	total: 107ms	remaining: 12s
18:	learn: 830.6564816	total: 112ms	remaining

169:	learn: 754.4291931	total: 994ms	remaining: 10.9s
170:	learn: 754.3294676	total: 1s	remaining: 10.9s
171:	learn: 754.1237546	total: 1.02s	remaining: 11s
172:	learn: 754.0258569	total: 1.03s	remaining: 11.1s
173:	learn: 753.7976505	total: 1.04s	remaining: 11.1s
174:	learn: 753.6788453	total: 1.05s	remaining: 11.1s
175:	learn: 753.5423188	total: 1.06s	remaining: 11.1s
176:	learn: 753.3777493	total: 1.06s	remaining: 11.1s
177:	learn: 753.1066810	total: 1.07s	remaining: 11.1s
178:	learn: 752.9089338	total: 1.08s	remaining: 11.1s
179:	learn: 752.7019909	total: 1.08s	remaining: 11.1s
180:	learn: 752.5469574	total: 1.09s	remaining: 11.1s
181:	learn: 752.4452640	total: 1.1s	remaining: 11.1s
182:	learn: 752.3306516	total: 1.1s	remaining: 11.1s
183:	learn: 752.1931323	total: 1.11s	remaining: 11.1s
184:	learn: 752.0823541	total: 1.11s	remaining: 11.1s
185:	learn: 751.9815533	total: 1.12s	remaining: 11.1s
186:	learn: 751.8734997	total: 1.13s	remaining: 11.1s
187:	learn: 751.7343851	total: 1.13

327:	learn: 733.0708235	total: 1.99s	remaining: 10.3s
328:	learn: 732.9025373	total: 2s	remaining: 10.3s
329:	learn: 732.7913036	total: 2.01s	remaining: 10.4s
330:	learn: 732.7059495	total: 2.02s	remaining: 10.4s
331:	learn: 732.5354362	total: 2.03s	remaining: 10.4s
332:	learn: 732.4145976	total: 2.03s	remaining: 10.4s
333:	learn: 732.3024892	total: 2.04s	remaining: 10.4s
334:	learn: 732.2286278	total: 2.04s	remaining: 10.3s
335:	learn: 732.0496115	total: 2.05s	remaining: 10.3s
336:	learn: 731.9078440	total: 2.06s	remaining: 10.3s
337:	learn: 731.7799692	total: 2.06s	remaining: 10.3s
338:	learn: 731.6287488	total: 2.07s	remaining: 10.3s
339:	learn: 731.4241038	total: 2.08s	remaining: 10.3s
340:	learn: 731.2940718	total: 2.08s	remaining: 10.3s
341:	learn: 731.1791335	total: 2.09s	remaining: 10.3s
342:	learn: 731.1067121	total: 2.09s	remaining: 10.3s
343:	learn: 730.9876310	total: 2.1s	remaining: 10.3s
344:	learn: 730.9236432	total: 2.1s	remaining: 10.3s
345:	learn: 730.7764393	total: 2.

496:	learn: 716.4222116	total: 2.98s	remaining: 9.19s
497:	learn: 716.3683243	total: 2.98s	remaining: 9.18s
498:	learn: 716.3221049	total: 2.99s	remaining: 9.17s
499:	learn: 716.2171920	total: 2.99s	remaining: 9.16s
500:	learn: 716.1224567	total: 3s	remaining: 9.15s
501:	learn: 716.0091418	total: 3s	remaining: 9.15s
502:	learn: 715.9089775	total: 3.01s	remaining: 9.14s
503:	learn: 715.8355765	total: 3.02s	remaining: 9.14s
504:	learn: 715.7445617	total: 3.03s	remaining: 9.14s
505:	learn: 715.6721324	total: 3.03s	remaining: 9.13s
506:	learn: 715.5745386	total: 3.04s	remaining: 9.13s
507:	learn: 715.4463277	total: 3.04s	remaining: 9.12s
508:	learn: 715.3542988	total: 3.05s	remaining: 9.12s
509:	learn: 715.2875391	total: 3.06s	remaining: 9.11s
510:	learn: 715.2040067	total: 3.06s	remaining: 9.11s
511:	learn: 715.1747833	total: 3.07s	remaining: 9.1s
512:	learn: 715.0814974	total: 3.07s	remaining: 9.09s
513:	learn: 714.9566669	total: 3.08s	remaining: 9.08s
514:	learn: 714.8683215	total: 3.08

662:	learn: 703.4505361	total: 3.98s	remaining: 8.2s
663:	learn: 703.4329268	total: 3.99s	remaining: 8.2s
664:	learn: 703.3584306	total: 3.99s	remaining: 8.2s
665:	learn: 703.2923373	total: 4s	remaining: 8.19s
666:	learn: 703.2146062	total: 4.01s	remaining: 8.19s
667:	learn: 703.1360349	total: 4.01s	remaining: 8.18s
668:	learn: 703.0561791	total: 4.02s	remaining: 8.18s
669:	learn: 702.9998157	total: 4.02s	remaining: 8.17s
670:	learn: 702.9074322	total: 4.03s	remaining: 8.16s
671:	learn: 702.8402614	total: 4.04s	remaining: 8.15s
672:	learn: 702.7409162	total: 4.04s	remaining: 8.15s
673:	learn: 702.6763674	total: 4.05s	remaining: 8.15s
674:	learn: 702.6059323	total: 4.06s	remaining: 8.15s
675:	learn: 702.5279921	total: 4.07s	remaining: 8.15s
676:	learn: 702.4763834	total: 4.08s	remaining: 8.16s
677:	learn: 702.3881079	total: 4.09s	remaining: 8.15s
678:	learn: 702.3282215	total: 4.09s	remaining: 8.14s
679:	learn: 702.2935259	total: 4.1s	remaining: 8.13s
680:	learn: 702.2242578	total: 4.1s

...

1302:	learn: 665.8167694	total: 7.8s	remaining: 4.35s
1303:	learn: 665.7626879	total: 7.8s	remaining: 4.34s
1304:	learn: 665.7028814	total: 7.81s	remaining: 4.34s
1305:	learn: 665.6576734	total: 7.82s	remaining: 4.34s
1306:	learn: 665.6093744	total: 7.83s	remaining: 4.33s
1307:	learn: 665.5703086	total: 7.83s	remaining: 4.32s
1308:	learn: 665.4896871	total: 7.84s	remaining: 4.32s
1309:	learn: 665.4550793	total: 7.85s	remaining: 4.31s
1310:	learn: 665.4061449	total: 7.85s	remaining: 4.3s
1311:	learn: 665.3635225	total: 7.86s	remaining: 4.3s
1312:	learn: 665.3031311	total: 7.86s	remaining: 4.29s
1313:	learn: 665.2584969	total: 7.87s	remaining: 4.29s
1314:	learn: 665.2195419	total: 7.87s	remaining: 4.28s
1315:	learn: 665.1660804	total: 7.87s	remaining: 4.27s
1316:	learn: 665.1564273	total: 7.88s	remaining: 4.26s
1317:	learn: 665.0874401	total: 7.88s	remaining: 4.26s
1318:	learn: 665.0763785	total: 7.89s	remaining: 4.25s
1319:	learn: 665.0225803	total: 7.89s	remaining: 4.25s
1320:	learn: 6

1467:	learn: 657.6208585	total: 8.79s	remaining: 3.37s
1468:	learn: 657.5579437	total: 8.8s	remaining: 3.36s
1469:	learn: 657.5154276	total: 8.81s	remaining: 3.36s
1470:	learn: 657.4766438	total: 8.83s	remaining: 3.35s
1471:	learn: 657.4319505	total: 8.83s	remaining: 3.35s
1472:	learn: 657.3939913	total: 8.84s	remaining: 3.34s
1473:	learn: 657.3542813	total: 8.85s	remaining: 3.34s
1474:	learn: 657.3172390	total: 8.85s	remaining: 3.33s
1475:	learn: 657.2918892	total: 8.86s	remaining: 3.32s
1476:	learn: 657.2415522	total: 8.86s	remaining: 3.32s
1477:	learn: 657.1900826	total: 8.87s	remaining: 3.31s
1478:	learn: 657.1154699	total: 8.87s	remaining: 3.31s
1479:	learn: 657.0756221	total: 8.88s	remaining: 3.3s
1480:	learn: 657.0316206	total: 8.88s	remaining: 3.29s
1481:	learn: 656.9746317	total: 8.89s	remaining: 3.29s
1482:	learn: 656.9225780	total: 8.89s	remaining: 3.28s
1483:	learn: 656.8763083	total: 8.89s	remaining: 3.27s
1484:	learn: 656.8313018	total: 8.9s	remaining: 3.27s
1485:	learn: 

1620:	learn: 650.7250041	total: 9.78s	remaining: 2.47s
1621:	learn: 650.6774897	total: 9.79s	remaining: 2.46s
1622:	learn: 650.6371429	total: 9.8s	remaining: 2.46s
1623:	learn: 650.5971965	total: 9.81s	remaining: 2.45s
1624:	learn: 650.5369578	total: 9.81s	remaining: 2.45s
1625:	learn: 650.4794492	total: 9.83s	remaining: 2.44s
1626:	learn: 650.4019328	total: 9.83s	remaining: 2.44s
1627:	learn: 650.3566617	total: 9.84s	remaining: 2.43s
1628:	learn: 650.3037218	total: 9.86s	remaining: 2.43s
1629:	learn: 650.2734460	total: 9.86s	remaining: 2.42s
1630:	learn: 650.2445991	total: 9.87s	remaining: 2.41s
1631:	learn: 650.2383472	total: 9.88s	remaining: 2.41s
1632:	learn: 650.2051835	total: 9.89s	remaining: 2.4s
1633:	learn: 650.1541092	total: 9.9s	remaining: 2.4s
1634:	learn: 650.1293466	total: 9.91s	remaining: 2.39s
1635:	learn: 650.0982913	total: 9.91s	remaining: 2.39s
1636:	learn: 650.0556137	total: 9.92s	remaining: 2.38s
1637:	learn: 650.0051741	total: 9.93s	remaining: 2.38s
1638:	learn: 6

1795:	learn: 642.9811766	total: 11.2s	remaining: 1.46s
1796:	learn: 642.9404820	total: 11.2s	remaining: 1.45s
1797:	learn: 642.8891193	total: 11.2s	remaining: 1.45s
1798:	learn: 642.8564921	total: 11.2s	remaining: 1.44s
1799:	learn: 642.8136370	total: 11.2s	remaining: 1.43s
1800:	learn: 642.7843968	total: 11.2s	remaining: 1.43s
1801:	learn: 642.7519302	total: 11.2s	remaining: 1.42s
1802:	learn: 642.7027887	total: 11.2s	remaining: 1.42s
1803:	learn: 642.6639545	total: 11.2s	remaining: 1.41s
1804:	learn: 642.6125604	total: 11.3s	remaining: 1.4s
1805:	learn: 642.5588379	total: 11.3s	remaining: 1.4s
1806:	learn: 642.5144317	total: 11.3s	remaining: 1.39s
1807:	learn: 642.4613005	total: 11.3s	remaining: 1.38s
1808:	learn: 642.4222523	total: 11.3s	remaining: 1.38s
1809:	learn: 642.3962387	total: 11.3s	remaining: 1.37s
1810:	learn: 642.3595005	total: 11.3s	remaining: 1.36s
1811:	learn: 642.3031944	total: 11.3s	remaining: 1.36s
1812:	learn: 642.2786002	total: 11.3s	remaining: 1.35s
1813:	learn:

1951:	learn: 636.4107696	total: 12.2s	remaining: 488ms
1952:	learn: 636.3338641	total: 12.2s	remaining: 482ms
1953:	learn: 636.3008159	total: 12.2s	remaining: 476ms
1954:	learn: 636.2653144	total: 12.2s	remaining: 469ms
1955:	learn: 636.1904918	total: 12.2s	remaining: 463ms
1956:	learn: 636.1325967	total: 12.3s	remaining: 457ms
1957:	learn: 636.0764295	total: 12.3s	remaining: 451ms
1958:	learn: 636.0421300	total: 12.3s	remaining: 444ms
1959:	learn: 636.0008494	total: 12.3s	remaining: 438ms
1960:	learn: 635.9555958	total: 12.3s	remaining: 432ms
1961:	learn: 635.8958521	total: 12.3s	remaining: 425ms
1962:	learn: 635.8496707	total: 12.3s	remaining: 419ms
1963:	learn: 635.8060090	total: 12.3s	remaining: 413ms
1964:	learn: 635.7813276	total: 12.3s	remaining: 407ms
1965:	learn: 635.7535324	total: 12.3s	remaining: 400ms
1966:	learn: 635.7156731	total: 12.3s	remaining: 394ms
1967:	learn: 635.6740014	total: 12.3s	remaining: 388ms
1968:	learn: 635.6465054	total: 12.3s	remaining: 381ms
1969:	lear

In [31]:
print("score:",model.score(X_train,y_train))

X2 = sm.add_constant(X_train)
est = sm.OLS(y_train, X2)
est2 = est.fit()
print(est2.summary())

score: 0.5887681805039715
                                 OLS Regression Results                                
Dep. Variable:     Time from Pickup to Arrival   R-squared:                       0.350
Model:                                     OLS   Adj. R-squared:                  0.349
Method:                          Least Squares   F-statistic:                     543.0
Date:                         Thu, 21 Nov 2019   Prob (F-statistic):               0.00
Time:                                 21:52:58   Log-Likelihood:            -1.7170e+05
No. Observations:                        21201   AIC:                         3.434e+05
Df Residuals:                            21179   BIC:                         3.436e+05
Df Model:                                   21                                         
Covariance Type:                     nonrobust                                         
                                    coef    std err          t      P>|t|      [0.025      0.9

In [ ]:
y_pred_ = model.predict(X_test)
sp["Time from Pickup to Arrival"] = y_pred_
sp.to_csv("zindi_0.csv",index=False)